# `chatXiv` - Chat with arXiv papers

In [1]:
!git clone https://github.com/nerdimite/chatxiv

Cloning into 'chatxiv'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 26 (delta 7), reused 20 (delta 4), pack-reused 0
Receiving objects: 100% (26/26), 24.29 KiB | 12.14 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:
cd chatxiv

/content/chatxiv


In [ ]:
pip install -r requirements.txt

In [4]:
import os
import json
import arxiv
import requests
import fitz
from pymupdf_rag import to_markdown
from typing import List, Tuple

import chromadb
from llama_index.core import Document, VectorStoreIndex, PromptTemplate
from llama_index.core.schema import NodeWithScore
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.ingestion import IngestionPipeline
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import MarkdownNodeParser
import chromadb.utils.embedding_functions as embedding_functions
from chat_utils import ChatLLM, ChatSession

In [ ]:
os.environ['OPENAI_API_KEY'] = "YOUR API KEY"

# Naive Q&A over a PDF Document

## Ingestion

### Load Data

In [ ]:
arxiv_client = arxiv.Client()

def download_pdf(paper_id: str, dirpath: str = './papers'):
    '''Downloads a paper from arXiv given its ID'''
    paper = next(arxiv.Client().results(arxiv.Search(id_list=[paper_id])))
    return paper.download_pdf(dirpath=dirpath), paper

In [ ]:
def load_arxiv_document(paper_id):
    '''Loads an arxiv paper as a Document object'''
    pdf_path, paper = download_pdf(paper_id, dirpath='./papers')
    paper_pdf = fitz.open(pdf_path)
    md_text = to_markdown(paper_pdf)

    return Document(
        doc_id=paper_id,
        text=md_text,
        metadata={
            'title': paper.title,
            'authors': ", ".join([auth.name for auth in paper.authors[:10]]),
            'published': paper.published.strftime('%Y-%m-%d'),
            'filepath': pdf_path,
        }
    )

In [ ]:
paper_id = "2312.00752"

In [ ]:
# Load the documents
documents = [
    load_arxiv_document(paper_id)
]

In [ ]:
# Breakdown the document into nodes
parser = MarkdownNodeParser()

nodes = parser.get_nodes_from_documents(documents)

### Create the VectorDB

In [ ]:
# Creating a Chroma VectorDB instance
db = chromadb.PersistentClient(path="./chroma_db")
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=os.getenv("OPENAI_API_KEY"),
                model_name="text-embedding-3-small"
            )
chroma_collection = db.get_or_create_collection("chatxiv", embedding_function=openai_ef)

# Create a Chroma VectorStore
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Create a Simple Document Store as well
docstore = SimpleDocumentStore()

# Load the embedding model
embed_model = OpenAIEmbedding(model="text-embedding-3-small", embed_batch_size=100)

In [ ]:
# Build the Embedding Pipeline
pipeline = ...

# load local cache (if it exists)
# pipeline.load("./pipeline_chatxiv")

# Run Pipeline

# save cache locally
pipeline.persist("./pipeline_chatxiv")

In [ ]:
# Create vector index for retrieval
index = ...

## Retrieval

In [ ]:
retriever = ...

In [ ]:
query_str = "Explain mamba"

In [ ]:
retrieved_nodes = ...

In [ ]:
retrieved_nodes[0].metadata

## Response Synthesis

In [ ]:
system_prompt = {
    'role': 'system',
    'content': """You are a Q&A bot. You are here to answer questions based on the context given.
You are prohibited from using prior knowledge and you can only use the context given. If you need
more information, please ask the user."""
}

In [ ]:
context_prompt = PromptTemplate(
"""Context information to answer the query is below.
---------------------
{context_str}
---------------------""")

def build_context_prompt(retrieved_nodes):
    context_str = "\n\n".join([r.node.get_content(metadata_mode='all') for r in retrieved_nodes])
    return context_prompt.format(
        context_str=context_str
    )

In [ ]:
llm = ChatLLM()
session = ChatSession(llm, system_prompt=system_prompt)

In [ ]:
# Add the context as a user message to the message history (thread)
session.thread.append({
    'role': 'user',
    'content': build_context_prompt(retrieved_nodes)
})

In [ ]:
response = session.chat("How does mamba work? Explain like I am 5")
print(response.content)

# Retrieval as a Tool

In [ ]:
from llama_index.core.tools import FunctionTool
from openai.types.chat.chat_completion_tool_message_param import ChatCompletionToolMessageParam

In [ ]:
def context_retrieval(search_query: str, top_k: int=3) -> Tuple[str, List[NodeWithScore]]:
    '''
    This function let's you semantically retrieve relevant context chunks from a given document based on a query.

    Arguments:
        query (str): The query to search for in the document. Based on the original user query, write a good search query
                     which is more logically sound to retrieve the relevant information from the document.
        top_k (int): The number of top chunks to retrieve from the document. Default is 3. You can increase this number if
                     you feel like you need more information. But ideally you should make multiple calls to retrieve different
                     topics of information.

    Returns:
        str: The top retrieved
        List[NodeWithScore]: A list of nodes with their scores. Use this to cite the information in the response.
    '''
    retriever = index.as_retriever(similarity_top_k=top_k)
    retrieved_nodes = retriever.retrieve(search_query)
    return build_context_prompt(retrieved_nodes), retrieved_nodes

In [ ]:
llm = ChatLLM()
session = ChatSession(llm, system_prompt=system_prompt)

In [ ]:
context_retrieval_tool = FunctionTool.from_defaults(fn=context_retrieval)

In [ ]:
available_functions = {
    "context_retrieval": context_retrieval,
}

In [ ]:
def conversation_turn(user_query):
    # Send the messages and available functions to the model
    # We will call our chat method from our session object which is already keeping track of the history
    response = session.chat(user_query,
                            model='gpt-3.5-turbo',
                            temperature=1,
                            max_tokens=512,
                            tools=[context_retrieval_tool.metadata.to_openai_tool()], tool_choice='auto')

    tool_calls = response.tool_calls

    # Check if the model wanted to call a function
    if tool_calls:
        # Call the function
        for tool_call in tool_calls:
            function_name = ...
            function_to_call = ...
            function_args = ...
            function_response, retrieved_nodes = function_to_call(**function_args)

            print(f'Retrieving with {tool_call}')

            # Add the function response to the message thread
            session.thread.append(
                ChatCompletionToolMessageParam(role='tool', tool_call_id=tool_call.id, name=function_name, content=function_response)
            )

        # Get a new response from the model where it can see the function response
        post_function_response = session.chat()

        return post_function_response

    return response

In [ ]:
response = conversation_turn("What are selective state spaces though?")
response.content